<a href="https://colab.research.google.com/github/datasett/nav-examples/blob/master/arbeidsforhold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup for editing in Colab

In [0]:
import sys

if 'dataverk' not in  sys.modules:
    !pip install dataverk

In [0]:
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# Imports

In [0]:
import dataverk as dv
import numpy as np
import pandas as pd
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as plio

# Get data

In [0]:
df = pd.read_csv('https://cc-p-nav.ckan.io/dataset/cf4af47c-fd3e-4a0b-bdda-962cff3caf4e/resource/fcf3b72e-4a25-4a5b-9f28-2864adbb057d/download/nye_arbeidsforhold_2017.csv', sep=';', low_memory=False)

In [0]:
#df_2016 = pd.read_csv('https://cc-p-nav.ckan.io/dataset/cf4af47c-fd3e-4a0b-bdda-962cff3caf4e/resource/c9aed227-6b4a-4064-b2bf-373d55697dcb/download/nye_arbeidsforhold_2016.csv', sep=';', low_memory=False)

In [0]:
#df_2015 = pd.read_csv('https://cc-p-nav.ckan.io/dataset/cf4af47c-fd3e-4a0b-bdda-962cff3caf4e/resource/b6330ad6-3cec-4710-9ca3-7167b86d44df/download/nye_arbeidsforhold_2015.csv', sep=';', low_memory=False)

In [0]:
df.head(5)

In [0]:
len(df)

In [0]:
df['yrkesfelt_navn'].unique()

In [0]:
df['arbeidssted_fylke_navn'].unique()

In [0]:
df_x = df[['yrkesfelt_navn','arbeidssted_fylke_navn','antall_nye_ansatte']]

In [0]:
df_x = df_x[(df_x['yrkesfelt_navn'] != 'I alt') & (df_x['arbeidssted_fylke_navn'] != 'I alt') & (df_x['antall_nye_ansatte'] != '*')].copy()

In [0]:
df_x['antall_nye_ansatte'] = df_x['antall_nye_ansatte'].astype(int)

In [0]:
df_p = df_x.pivot_table(index='yrkesfelt_navn', 
           columns='arbeidssted_fylke_navn', 
           values='antall_nye_ansatte',
           aggfunc=np.sum
          ).reset_index().set_index('yrkesfelt_navn')

In [0]:
df_p.dtypes

In [0]:
df_p.index.tolist()

In [0]:
data = [go.Surface(z=df_p.values.tolist(),
                   y=df_p.columns[1:].tolist(),
                   x=df_p.index.tolist(),
                   colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)',
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230, 230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict(x=1, y=1, z=0.7),
        aspectmode = 'manual'
    )
)

updatemenus=list([
    dict(
        buttons=list([   
            dict(
                args=['type', 'surface'],
                label='3D Surface',
                method='restyle'
            ),
            dict(
                args=['type', 'heatmap'],
                label='Heatmap',
                method='restyle'
            )             
        ]),
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

annotations = list([
    dict(text='Trace type:', x=0, y=1.085, yref='paper', align='left', showarrow=False)
])
layout['updatemenus'] = updatemenus
layout['annotations'] = annotations

fig = go.Figure(data,layout)

go.FigureWidget(data,layout)

In [0]:
if 'google.colab' in sys.modules:
    def enable_plotly_in_cell():
      import IPython
      from plotly.offline import init_notebook_mode
      display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
      '''))
      init_notebook_mode(connected=False)

    from plotly.offline import iplot
    enable_plotly_in_cell()
    iplot(data)

# Datapackage

In [0]:
readme = """
# Arbeidsforhold

"""

metadata = {
'store': 'https://raw.githubusercontent.com/datasett/nav-examples/master',
'title': 'Arbeidsforhold',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'auth': 'Dataverk',
'description':'Arbeidsforhold',
'name':'Arbeidsforhold',
'source':'Dataverk',
'keywords':['test'],
'provenance':'Dataverk',
'publisher': 'Dataverk',
'project': 'mesiqi', 
'user':'dataverk',
'bucket_name': 'dataparcel',
'dataset':'arbeidsforhold'
}

In [0]:
dp = dv.Datapackage(metadata)

## Add views

In [0]:
description = '''
Nye ansatte i 2017 

'''

dp.add_view(
    spec_type='plotly',
    name='Nye ansatte i 2017',
    title='Nye ansatte i 2017',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig)
)

# Save data and publish datapackage in elastic search index

In [0]:
client = dv.Client()
client.publish(dp)